__Example 7.15.__(Smith) Consider the spring model

$\ddot{z} + C\dot{z} + Kz = 0$

$z(0) = 2$, $\dot{z}(0) = -C$

with displacement observations so that

$y = \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} z \\ \dot{z} \end{bmatrix} = z$.

The solution is 

$z(t) = 2e^{-Ct/2}\cos{\left( \sqrt{K-C^2/4} t\right)}$.

Suppose that $C$ is unknown. $q=C$ and $p=1$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#from IPython.display import Image
from scipy import stats
import numpy as np

In [ ]:
K = 20.5 # known
C0 = 1.5 #<---- fixed but unknown value 
p = 1 #number of unknown parameters

In [ ]:
def f(t,C,K):
    # Closed form solution at time t for parameters C and K
    return 2*np.exp(-C*t/2)*np.cos(np.sqrt(K-C**2/4)*t)

### Generate "observations" $y$ and plot them

In [ ]:
n = 501 #number of observations
t = np.linspace(0, 5, n) #
ytrue = f(t,C0,K)
sigma = 0.1 #<-------- true noise
y = ytrue + np.random.normal(0, sigma, n)
fig, ax  = plt.subplots()
ax.plot(t,ytrue);
ax.plot(t, y,'ro',markersize=1)
plt.xlabel('Time'),plt.ylabel('displacement')
plt.legend(['Truth','Observations'])

### Find the C that minimizes the sum squared error

In [ ]:
def SSE(C,y):
    return sum( (f(t,C,K)-y)**2 )

In [ ]:
from scipy import optimize
C_OLS=optimize.fmin(SSE,1,args=(y,))
C_OLS

#### Residules

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t, f(t,C_OLS,K)-y,'.');
plt.title('Residuals $\hat\epsilon$')

#### Estimate error variance

In [ ]:
sigma_hat = np.sqrt(SSE(C_OLS,y)/(n-p))
sigma_hat

Closed form for the elements of the sensitivity matrix $X$. (Here a vector because $p=1$)
$$\dfrac{\partial y}{\partial C} = e^{-C t/2} \left[ \dfrac{Ct}{\sqrt{4K-C^2}} \sin \left( t \sqrt{K - C^2/4}\right) - t \cos \left( t \sqrt{K - C^2/4}\right) \right]$$

In [ ]:
C = C_OLS
w = np.sqrt(K - C**2/4)
X = np.exp(-C*t/2)*(C*t/np.sqrt(4*K-C**2)*np.sin(w*t)-t*np.cos(w*t))

$V = \sigma_C^2 = \sigma_0^2 [ X(q)^T X(q) ]^{-1}$, $X(q) = \left[ \dfrac{\partial y}{\partial C} (t_1,q),\dots,\dfrac{\partial y}{\partial C} (t_N,q) \right]^T$

In [ ]:
V = sigma_hat**2/np.dot(X,X)
np.sqrt(V)

95% confidence interval is:

In [ ]:
alpha = 0.05
Interval_normal = C_OLS + stats.norm.ppf(1-alpha)*np.sqrt(V)*np.array([-1,1])
Interval_t = C_OLS + stats.t.ppf(1-alpha,df=n-1)*np.sqrt(V)*np.array([-1,1])
Interval_t, Interval_normal

Note that the _interval_ is random. How often is the true value in the interval?
If we repeat the problem (not possible in practice), we can check how often the true value is within the interval.

In [ ]:
mc = 1000
Csamples = np.zeros(mc)
InInterval = np.zeros(mc)
Y = np.zeros((n,mc))
for imc in range(0,mc):
    ysample = f(t,C0,K) + np.random.normal(0, sigma, n)
    Y[:,imc] = ysample
    C = optimize.fmin(SSE,C_OLS,args=(ysample,),disp=0)
    Csamples[imc] = C
    sigma_hat = np.sqrt(SSE(C,ysample)/(n-p))
    w = np.sqrt(K - C**2/4)
    Xs = np.exp(-C*t/2)*(C*t/np.sqrt(4*K-C**2)*np.sin(w*t)+t*np.cos(w*t))
    Vs = sigma_hat**2/np.dot(Xs,Xs)
    InInterval[imc] = (C0 < C+1.96*np.sqrt(Vs)) & (C0 > C-1.96*np.sqrt(Vs))

In [ ]:
np.mean(InInterval)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(Csamples);

In [ ]:
fig, ax  = plt.subplots()
plt.hist(Csamples, color = 'blue', edgecolor = 'black',bins = int(180/5))

In [ ]:
np.mean(Csamples)

In [ ]:
np.std(Csamples),np.sqrt(V)

#### Compute sensitivity matrix by finite differences of closed form solution.

In [ ]:
delta_C = 0.01
X_fd =  (f(t,C_OLS+delta_C,K) - f(t,C_OLS,K))/delta_C
V_fd = sigma_hat**2/np.dot(X_fd,X_fd)
np.sqrt(V_fd)

## What if we don't know the solution?

$$ \frac{d}{dt} \begin{bmatrix} z \\ \dot{z} \end{bmatrix} = \begin{bmatrix} \dot{z} \\ -C \dot{z} -Kz \end{bmatrix}\,, \qquad \begin{bmatrix} z \\ \dot{z} \end{bmatrix} = \begin{bmatrix} 2 \\ -C \end{bmatrix}$$

In [ ]:
from scipy.integrate import odeint

In [ ]:
def dU_dt(U, t, C, K):
    # Here U is a vector such that y=U[0] and z=U[1]. This function should return [y', z']
    return [U[1], -C*U[1] - K*U[0]]
U0 = [2, -C]
Us = odeint(dU_dx, U0, t, args=(C0,K))
y_numerical = Us[:,0]

In [ ]:
plt.xlabel("t")
plt.ylabel("z(t)")
plt.title("Damped harmonic oscillator")
plt.plot(t,ytrue);
plt.plot(t,y_numerical);

In [ ]:
plt.xlabel("t")
plt.ylabel("z(t)")
plt.title("Damped harmonic oscillator")
plt.plot(t,ytrue-y_numerical);
np.sqrt(np.mean((ytrue-y_numerical)**2))

In [ ]:
def SSE2(C,y):
    wrk = odeint(dU_dt, U0, t, args=(C,K))
    return sum( (wrk[:,0]-y)**2 )

In [ ]:
C_OLS_numerical=optimize.fmin(SSE2,1,args=(y,))
C_OLS,C_OLS_numerical

In [ ]:
wrk = odeint(dU_dt, U0, t, args=(C_OLS_numerical+delta_C,K)) - odeint(dU_dx, U0, t, args=(C_OLS_numerical,K))
X_fd2 =  wrk[:,0]/delta_C
V_fd2 = sigma_hat**2/np.dot(X_fd2,X_fd2)
np.sqrt(V_fd)

## Sensitivity equations

$$ \frac{d}{dt} \begin{bmatrix} z_C \\ \dot{z}_C \end{bmatrix} = \begin{bmatrix} \dot{z}_C \\ -C \dot{z}_C -Kz_C - \dot{z} \end{bmatrix}\,, \qquad \begin{bmatrix} z_C \\ \dot{z}_C \end{bmatrix} = \begin{bmatrix} 0 \\ -1 \end{bmatrix}$$

In [ ]:
def zdot(t,C,K):
    U0 = [2, -C]
    wrk = odeint(dU_dt, U0, t, args=(C,K))
    return wrk[:,1]

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t,zdot(t,C,K));
ax.plot(t,(f(t+0.001,C,K)-f(t,C,K))/0.001);

In [ ]:
def dUC_dt(U, t, C, K):
    # Here U is a vector such that y=U[0] and z=U[1]. This function should return [y', z']
    dt = 0.01                 
    return [U[1], -C*U[1] - K*U[0] - (f(t+dt,C,K)-f(t,C,K))/dt]
#    return [U[1], -C*U[1] - K*U[0] - zdot(t,C,K)]

UC0 = [0, -1]
wrk = odeint(dUC_dt, UC0, t, args=(C_OLS,K))
Xs = wrk[:,0]

In [ ]:
Vs = sigma_hat**2/np.dot(Xs,Xs)
np.sqrt(Vs)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(X);
ax.plot(X_fd);
ax.plot(X_fd2);
ax.plot(Xs);
plt.legend(['Exact','Finite difference of exact solution','Finite difference of numerical solution','Sensitivity equation'])

# BUG